<a href="https://colab.research.google.com/github/Vinijasmi20V/Real-time-Weather-Prediction/blob/main/Weather_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1: Import Libraries

In [ ]:
import requests #This Library helps us to fetch data from API
import pandas as pd #for handling and analysing data
import numpy as np #for numerical operations
from sklearn.model_selection import train_test_split #to split data into training and testing sets
from sklearn.preprocessing import LabelEncoder #to convert categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression
from sklearn.metrics import mean_squared_error #to measure accuracy of prediction
from datetime import datetime, timedelta #to handle date and time
import pytz

In [ ]:
API_KEY='a32c8fe9f2e4baee6cd140835d85b610' #REplace  with your actual API Keys
BASE_URL='https://api.openweathermap.org/data/2.5/' #base url for making API requests

1. Fetch cuuent weather data

In [ ]:
def get_current_weather(city):
    url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #construct the API request url
    response=requests.get(url) #send the request to the API
    data=response.json() #parse the response as JSON
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min':round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']) ,
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure':data['main']['pressure'],
        'wind_gust_speed': data['wind']['speed']

    }


2. Read Historical Data

In [ ]:
def read_historical_data(weather_csv):
   df=pd.read_csv(weather_csv) # load csv file into dataframe
   df=df.dropna() #remove rows with missing values
   df=df.drop_duplicates()
   return df

3. Prepare data for training

In [ ]:
def prepare_data(data):
    le=LabelEncoder()
    data['WindGustDir']=le.fit_transform(data['WindGustDir'])
    data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])
    X=data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']]
    y=data['RainTomorrow'] #target variable



    return X,y,le

**Train Rain Prediction Model**
   

In [ ]:
def train_rain_model(x,y):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(x_train, y_train)

  y_pred = model.predict(x_test)

  print("Mean Squared Error for Rain Model")

  print(mean_squared_error(y_test, y_pred))

  return model





**5.Prepare regression data**

In [ ]:
def prepare_regression_data(data, feature):
  x,y = [], []

  for i in range(len(data) - 1):
    x.append(data[feature].iloc[i])

    y.append(data[feature].iloc[i+1])

  x = np.array(x).reshape(-1, 1)
  y = np.array(y)
  return x,y

**Train Regression Model**

In [ ]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(x, y)
  return model

**Predict Future**

In [ ]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions[1:]

**Weather Analysis Function**

In [ ]:
def weather_view():
    city = input('Enter any city name: ')
    current_weather = get_current_weather(city)

    # Load historical data
    historical_data = read_historical_data('/content/weather.csv')

    # Prepare and train the rain prediction model
    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    # Map wind direction to compass points
    wind_deg = current_weather.get('wind_gust_dir',0) % 360  # Get wind_gust_dir, default to 0 if not found
    compass_points = {
        ("N", 0, 11.25), ("NNE", 11.5, 33.75), ("NE", 33.75, 56.25), ("ENE", 56.25, 78.75),
        ("E", 78.75, 101.25), ("ESE", 101.25, 123.75), ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75),
        ("S", 168.75, 191.25), ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25), ("NNW", 326.25, 348.75)
    }
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)  # Added a default value (None)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,  # Fixed the missing comma
        'WindGustSpeed': current_weather.get('wind_gust_speed',0),
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }
    current_df = pd.DataFrame([current_data])

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression model for temperature and humidity
    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')
    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)

    # Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_hum = predict_future(hum_model, current_weather['humidity'])

    # Prepare time for future predictions
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]  # Fixed strftime

    # Display results
    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}")
    print(f"Feels Like: {current_weather['feels_like']}")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction : {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Prediction:")
    for time, humidity in zip(future_times, future_hum):
        print(f"{time}: {round(humidity, 1)}%")

weather_view()


Enter any city name: kanyakumari
Mean Squared Error for Rain Model
0.1506849315068493
City: kanyakumari, IN
Current Temperature: 28
Feels Like: 30
Minimum Temperature: 28°C
Maximum Temperature: 28°C
Humidity: 62%
Weather Prediction: broken clouds
Rain Prediction : Yes

Future Temperature Predictions:
08:00: 27.7°C
09:00: 22.8°C
10:00: 25.4°C
11:00: 24.7°C
12:00: 23.4°C

Future Humidity Prediction:
08:00: 56.5%
09:00: 40.7%
10:00: 52.9%
11:00: 51.0%
12:00: 46.6%
